# Análisis de oferta

Solución para hacer análisis de oferta del modelo de abastecimiento estratégico.

In [1]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Paquetes de consulta a traves de Socrata

from sodapy import Socrata
identificador_columnas=pd.read_csv('datos/identificador_columnas.csv')
identificador_columnas=identificador_columnas[['Unificado','API_DA_S_I', 'API_DA_S_II']]
identificador_columnas=identificador_columnas.dropna()
identificador_columnas=identificador_columnas.set_index('Unificado')
identificador_columnas.drop('Fuente',axis=0,inplace=True)
identificador_datos={'SECOP I':'f789-7hwg','SECOP II':'jbjy-vk9h','TVEC':'rgxm-mmea'}

### Definición de cliente para Socrata

cliente=Socrata('www.datos.gov.co',None,timeout=1000000)


In [3]:
identificador_columnas['API_DA_S_I']['ID Contrato']

'numero_de_contrato'

In [ ]:
query_="""
SELECT 
    count({}) as Cantidad
LIMIT 
    1000000
""".format(identificador_columnas.['API_DA_S_I']['ID Contrato'])

In [3]:


### Definición de funciones para consulta de datos

query_SI="""SELECT
            {0}
        LIMIT
            15000000
        """.format(','.join(identificador_columnas['API_DA_S_I']))

query_SII="""SELECT
            {0}
        LIMIT
            15000000
        """.format(','.join(identificador_columnas['API_DA_S_II']))

results_SI=cliente.get(identificador_datos['SECOP I'],query=query_SI)
results_SI=pd.DataFrame.from_records(results_SI)
results_SI.columns=identificador_columnas.index


results_SII=cliente.get(identificador_datos['SECOP II'],query=query_SII)
results_SII=pd.DataFrame.from_records(results_SII)
results_SII.columns=identificador_columnas.index


total_SECOP=pd.concat([results_SI,results_SII],axis=0)


: 

: 

In [ ]:
total_SECOP.head(10)


In [ ]:
total_SECOP.columns